<a href="https://colab.research.google.com/github/jenny550/mit_futuremakers_create-a-thon_2023/blob/main/wildfires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Wildfire Image Detection

In [ ]:
!pip install ipyleaflet  # module source: jupyter notebook map feature module: #https://ipyleaflet.readthedocs.io/en/latest/index.html
!pip install ipinfo #source: https://github.com/ipinfo/python
!pip install googlemaps

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from PIL import Image
import os
from google.colab import drive
import ee


drive.mount('/content/gdrive')
with open('/content/gdrive/My Drive/file.txt', 'w') as f:
    f.write('content')

In [ ]:
ee.Authenticate()
ee.Initialize()
#verification code: click link and accept account

In [ ]:
import requests
import json
from geopy.geocoders import Nominatim

#easiest way to get longitude and latitude from user. Inf ututre situations, ethics and user permission will be requested.


app = Nominatim(user_agent="tutorial")
location = app.geocode("Massachusetts, United States").raw
lat, lng = location['lat'], location['lon']
print(f"""
     latitude: {lat}
     longitude: {lng}
      """)

In [ ]:
#get map source: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#get_a_static_map
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker

map = Map(
          basemap = basemaps.Esri.WorldImagery,
          center = [lat, lng],
          zoom = 12,

          )
#map.add_layer(Marker(location=(long_lat)))

map

#Model Architecture

In [ ]:
#history =  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                  #   validation_data = #Enter Code ,
                    # verbose=1)

In [ ]:
#loading images into arrays

train_fire_arr = [np.array( Image.open(f'/content/gdrive/MyDrive/WildFires_SureStart_Comp/training/fire/{x}').resize((128,128)) ) for x in os.listdir('/content/gdrive/MyDrive/WildFires_SureStart_Comp/training/fire')] #training fie imgs
train_noFire_arr = [np.array(Image.open(f'/content/gdrive/MyDrive/WildFires_SureStart_Comp/training/nofire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/gdrive/MyDrive/WildFires_SureStart_Comp/training/nofire')] #training noFire imgs

test_fire_arr = [np.array(Image.open(f'/content/gdrive/MyDrive/WildFires_SureStart_Comp/testing/fire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/gdrive/MyDrive/WildFires_SureStart_Comp/testing/fire')] #testing fire imgs
test_noFire_arr = [np.array(Image.open(f'/content/gdrive/MyDrive/WildFires_SureStart_Comp/testing/noFire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/gdrive/MyDrive/WildFires_SureStart_Comp/testing/noFire')] #testing noFire imgs

train = np.array(train_fire_arr + train_noFire_arr)
print('train: fire length: ', len(train_fire_arr))
print('train: noFire length: ', len(train_noFire_arr))
print('train: fire + noFire length: ', len(train))

test = np.array(test_fire_arr + test_noFire_arr)
print('test: fire length: ', len(test_fire_arr))
print('test: noFire length: ', len(test_noFire_arr))
print('test: fire + noFire length: ', len(test))

#print(train_fire_arr[0])
#plt.imshow(train[1800])

test[0].shape # each img is 250 x 250

In [ ]:
# x_train = images of fires
#labels = {0: no fire, 1.0: fire}
#y_train = all labels of fire, hint: {1.0}

x_train, y_train = train, np.concatenate((np.ones(len(train_fire_arr)), np.zeros(len(train_noFire_arr))), axis = 0)
x_test, y_test = test, np.concatenate((np.ones(len(test_fire_arr)), np.zeros(len(test_noFire_arr))), axis = 0)
idx = -1

x_train = x_train / 255.0
x_test = x_test / 255.0

print(y_train[idx])
print(x_train[idx])
plt.imshow(x_train[idx])

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
train = ImageDataGenerator(rescale=1/255) # conguring training dataset frame, pixel / 255
test = ImageDataGenerator(rescale=1/255) # conguring training dataset frame, pixel / 255

train_dataset = train.flow_from_directory("/content/gdrive/MyDrive/WildFires_SureStart_Comp/training",
                                          target_size=(128,128),
                                          batch_size = 32,
                                          class_mode='binary') # further configuring dataset frame, already loaded with image contents

test_dataset = test.flow_from_directory("/content/gdrive/MyDrive/WildFires_SureStart_Comp/testing",
                                          target_size=(128,128),
                                          batch_size =32,
                                          class_mode = 'binary')  # further configuring dataset frame, already loaded with image contents

In [ ]:
train_dataset.class_indices #print out class indicies: 0 for {fire}, 1 for {no fire }

In [ ]:
#strides increased for speed
stride = (2,2)

#padding added to ensure edges of inputs are also processed
padd = 'same'

#model source: https://www.kaggle.com/code/andersonblanco/forest-fire-detection-using-cnn/edit
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3), strides = stride, padding = padd))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu', strides = stride, padding = padd))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu', strides = stride, padding = padd))
model.add(keras.layers.MaxPool2D(2,2))
#model.add(keras.layers.Conv2D(128,(3,3),activation='relu', strides = stride, padding = padd)) #removing these layers, caused the model to increase accuracy and speed. As we saw these layers as uneccessarry.
#model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
#visualizing filters: weights

filters, biases = model.layers[0].get_weights()
min, max = filters.min(), filters.max()
filters = (filters - min) / (filters - max)

n_filters, ix = 6,1
fig, ax = plt.subplots(n_filters, 3, figsize=(10,10))
for i in range(n_filters):
  f = filters[:,:,0,i]

  for j in range(3):
    ax[i,j].imshow(f[:,:])
    ix += 1

In [ ]:
#display first image in x_test
plt.imshow(x_test[0])


In [ ]:
#feature maps: input looks after applying filters(weights)

#refining model to output after first layer:
x_model = keras.models.Sequential(model.get_layer(index = 0))
featureMaps = x_model.predict(x_test)
ix = 1
fig, ax = plt.subplots(8,4, figsize = (10,10))

for i in range(8):
  for j in range(4):
    ax[i, j].imshow(featureMaps[0, :, :, ix-1])
    ix += 1



In [ ]:
#y_train.resize(1832, 1)
#print(y_train[2])
fitted = model.fit(train_dataset, validation_data=test_dataset, epochs = 10)

In [ ]:
#prediction examples:
# 0 == Fire, 1 == No Fire
fires = test_fire_arr[0:8]
noFires = test_noFire_arr[8:16]
totals = fires + noFires

idx = 0
fig, ax = plt.subplots(4,4, figsize = (10,12), squeeze=False )

for row in range(4):
  for column in range(4):
    a = ax[row, column]
    X = np.expand_dims(totals[idx], axis = 0)
    predVal = model.predict(X)
    a.imshow(totals[idx])
    if predVal == 1:
      a.set_title('No Fire', fontsize = 15)
    else:
      a.set_title('Fire', fontsize = 15)
    idx += 1


In [ ]:
#training vs testing loss and accuracy
plt.plot(fitted.history['accuracy'], label='accuracy')
plt.plot(fitted.history['val_loss'], label='val_loss')
plt.plot(fitted.history['loss'], label='loss')
plt.legend()

#the graph shows the

In [ ]:
#accuracy plotting
plt.plot(fitted.history['accuracy'], label='accuracy')
plt.plot(fitted.history['val_accuracy'], label='val_accuracy')
plt.legend()

In [ ]:
#predictions
def predict(img):
  X = np.expand_dims(img, axis = 0)
  predVal = model.predict(X)
  print(predVal)
  plt.imshow(img)


  if predVal == 1:
    plt.xlabel('No Fire', fontsize = 34)
  else:
    plt.xlabel('Fire', fontsize = 34)

In [ ]:
#test_fire_arr
#test_noFire_arr
predict(test_fire_arr[-15])

In [ ]:
#visualizing activation:
def visualize_activation(model,activations,activation_model):
  layer_names = []
  for layer in model.layers[0:9]:
      layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot

  images_per_row = 16

  for layer_name, layer_activation in zip(layer_names, activations):       # Displays the feature maps
      n_features = layer_activation.shape[-1]                              # Number of features in the feature map
      size = layer_activation.shape[1]                                     # The feature map has shape (1, size, size, n_features).
      n_cols = n_features // images_per_row                                # Tiles the activation channels in this matrix
      display_grid = np.zeros((size * n_cols, images_per_row * size))
      for col in range(n_cols):                                            # Tiles each filter into a big horizontal grid
          for row in range(images_per_row):
              channel_image = layer_activation[0, :, :, col * images_per_row + row]
              channel_image -= channel_image.mean()                        # Post-processes the feature to make it visually palatable
              channel_image /= channel_image.std()
              channel_image *= 64
              channel_image += 128
              channel_image = np.clip(channel_image, 0, 255).astype('uint8')
              display_grid[col * size : (col + 1) * size,row * size : (row + 1) * size] = channel_image
      scale = 1. / size
      plt.figure(figsize=(scale * display_grid.shape[1],
                          scale * display_grid.shape[0]))
      plt.title(layer_name)
      plt.grid(False)
      plt.imshow(display_grid, aspect='auto', cmap='viridis')


img = np.array(test[0])
img.resize(1, 128,128, 3)
print(img.shape)
# Storing outputs from each layer of the model
layer_outputs = [layer.output for layer in model.layers[0:9]]

activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

# Getting activations (predictions) of a test image
activations  =  activation_model.predict(img)

visualize_activation(model,activations,activation_model)

In [ ]:
img = np.array(test[0])
img.resize(1, 128,128, 3)
print(img.shape)